In [1]:
import numpy as np
from ultralytics import YOLO
import math
import cv2
import cvzone
from sort import *
import time

model = YOLO("../Yolo-Weights/yolov8l.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


In [2]:
cap = cv2.VideoCapture("D:/study/CV/CK/Object-Detection-101/Videos/cars.mp4")  # For Video

mask = cv2.imread("D:/study/CV/CK/Object-Detection-101/Project 1 - Car Counter/mask1.png")

In [37]:
mask.shape

(720, 1280, 3)

In [6]:
# Khởi tạo đối tượng tracker với các tham số
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# Định nghĩa giới hạn theo tọa độ [x1, y1, x2, y2]
limits = [300, 340, 700, 360]
totalCount = []  # Danh sách lưu trữ các đối tượng đã đếm

while True:
    # Đọc frame từ camera
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)

    # Gọi mô hình để nhận diện đối tượng trong vùng quan tâm
    results = model(imgRegion, stream=True)

    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Lấy thông tin về hộp giới hạn (bounding box)
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Lấy độ tin cậy (confidence) và tên lớp của đối tượng
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            # Lọc ra các đối tượng là xe và có độ tin cậy lớn hơn 0.3
            if currentClass in ["car", "truck", "bus", "motorbike"] and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Cập nhật thông tin tracking của đối tượng
    resultsTracker = tracker.update(detections)

    # Vẽ đường giới hạn trên ảnh
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)

    # Hiển thị và đếm các đối tượng theo ID
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1

        # Vẽ khung và ID của đối tượng
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)

        # Vẽ điểm trung tâm của đối tượng
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        # Kiểm tra xem đối tượng có vượt quá giới hạn không
        if limits[0] < cx < limits[2] and limits[1] - 15 < cy < limits[1] + 15:
            if totalCount.count(id) == 0:
                totalCount.append(id)
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)

    # Hiển thị số lượng đối tượng đã đếm
    cvzone.putTextRect(img, f' Count: {len(totalCount)}', (50, 50))
    # cv2.putText(img, str(len(totalCount)), (255, 100), cv2.FONT_HERSHEY_PLAIN, 5, (50, 50, 255), 8)

    # Hiển thị ảnh và chờ nhấn phím
    cv2.imshow("Image", img)
    cv2.waitKey(1)


0: 384x640 3 cars, 619.6ms
Speed: 0.0ms preprocess, 619.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 614.1ms
Speed: 0.0ms preprocess, 614.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 529.2ms
Speed: 2.6ms preprocess, 529.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 521.0ms
Speed: 0.4ms preprocess, 521.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 523.4ms
Speed: 3.7ms preprocess, 523.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 533.5ms
Speed: 0.0ms preprocess, 533.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 538.8ms
Speed: 0.0ms preprocess, 538.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 520.3ms
Speed: 2.0ms preprocess, 520.3ms inference, 6.0ms postprocess per

ZeroDivisionError: float division by zero

In [5]:
# Khởi tạo đối tượng tracker với các tham số
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)
start_time = {}
# Định nghĩa giới hạn theo tọa độ [x1, y1, x2, y2]
first_limit = [300, 340, 700, 340]
last_limit = [0, 597, 673, 597]

counter = []

while True:
    # Đọc frame từ camera
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)

    # Gọi mô hình để nhận diện đối tượng trong vùng quan tâm
    results = model(imgRegion, stream=True)

    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Lấy thông tin về hộp giới hạn (bounding box)
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Lấy độ tin cậy (confidence) và tên lớp của đối tượng
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            # Lọc ra các đối tượng là xe và có độ tin cậy lớn hơn 0.3
            if currentClass == 'car' and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Cập nhật thông tin tracking của đối tượng
    resultsTracker = tracker.update(detections)

    # Vẽ đường giới hạn trên ảnh
    cv2.line(img, (first_limit[0], first_limit[1]), (first_limit[2], first_limit[3]), (0, 0, 255), 5)

    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1

        # Vẽ khung của đối tượng
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cx, cy = x1 + w // 2, y1 + h // 2

        # Lấy thông tin vận tốc (giả sử vận tốc là 0, bạn cần thay thế bằng hàm tính toán vận tốc thực tế)
        if first_limit[1] < cy < first_limit[3]:
            start_time[id] = time.time()

        if id in start_time and last_limit[1] < cy < last_limit[3]:
            elapsed_time = time.time() - start_time[id]
            if counter.count(id) == 0:
                # Tính toán và hiển thị vận tốc khi đi xuống
                distance = 10  # meters
                a_speed_ms = distance / elapsed_time
                a_speed_kh = a_speed_ms * 3.6
                cvzone.putTextRect(img, f'V: {a_speed_kh:.2f} km/h', (x2, y2), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    # Hiển thị đường giới hạn trên ảnh
    cv2.line(img, (last_limit[0], last_limit[1]), (last_limit[2], last_limit[3]), (255, 0, 0), 5)

    # Hiển thị ảnh và chờ nhấn phím
    cv2.imshow("Image", img)
    cv2.waitKey(1)


0: 384x640 1 car, 2 trucks, 513.4ms
Speed: 3.2ms preprocess, 513.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 595.3ms
Speed: 2.5ms preprocess, 595.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 535.4ms
Speed: 3.9ms preprocess, 535.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 505.9ms
Speed: 0.0ms preprocess, 505.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 478.2ms
Speed: 7.8ms preprocess, 478.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 450.5ms
Speed: 0.0ms preprocess, 450.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 485.9ms
Speed: 0.0ms preprocess, 485.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 468.1ms
Speed: 2.8ms preprocess, 468.1ms inf

KeyboardInterrupt: 

In [ ]:
# Khởi tạo đối tượng tracker với các tham số
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# Định nghĩa giới hạn theo tọa độ [x1, y1, x2, y2]
first_limit = [300, 340, 700, 340]
last_limit = [0, 597, 673, 597]

while True:
    # Đọc frame từ camera
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)

    # Gọi mô hình để nhận diện đối tượng trong vùng quan tâm
    results = model(imgRegion, stream=True)

    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Lấy thông tin về hộp giới hạn (bounding box)
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Lấy độ tin cậy (confidence) và tên lớp của đối tượng
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            # Lọc ra các đối tượng là xe và có độ tin cậy lớn hơn 0.3
            if currentClass in ["car", "truck", "bus", "motorbike"] and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Cập nhật thông tin tracking của đối tượng
    resultsTracker = tracker.update(detections)

    # Vẽ đường giới hạn trên ảnh
    cv2.line(img, (first_limit[0], first_limit[1]), (first_limit[2], first_limit[3]), (0, 0, 255), 5)

    # Hiển thị thông tin vận tốc
    for result in resultsTracker:
        x1, y1, x2, y2, _ = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1

        # Vẽ khung của đối tượng
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))

        # Lấy thông tin vận tốc (giả sử vận tốc là 0, bạn cần thay thế bằng hàm tính toán vận tốc thực tế)
        speed = 0

        # Hiển thị thông tin vận tốc
        speed_text = f'{speed} km/h'
        cvzone.putTextRect(img, speed_text, (max(0, x1), max(35, y1)),
                        scale=1, thickness=2, offset=10, colorR=(255, 0, 0))

    # Hiển thị ảnh và chờ nhấn phím
    cv2.imshow("Image", img)
    cv2.waitKey(1)


In [13]:
# Khởi tạo đối tượng tracker với các tham số
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# Định nghĩa giới hạn theo tọa độ [x1, y1, x2, y2]
first_limit = [300, 340, 700, 340]
last_limit = [0, 597, 673, 597]

# Danh sách lưu trữ các đối tượng đã đếm
total_objects = {}

def calculate_speed(x1, y1, x2, y2, time_interval = 1):
    # Thực hiện tính toán vận tốc dựa trên tọa độ (x1, y1) và (x2, y2)
    # Bạn có thể sử dụng các phương pháp tính toán vận tốc, ví dụ: khoảng cách / thời gian
    # Đây chỉ là một hàm giả định, bạn cần thay thế bằng cách tính toán thực tế của bạn
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    # Giả sử thời gian giữa hai frame là 1 giây
    speed = distance / time_interval
    return speed

def update_object_info(object_id, timestamp, speed):
    if object_id not in total_objects:
        total_objects[object_id] = {'speed': None, 'timestamp': None}
        total_objects[object_id]['speed'] = speed
        total_objects[object_id]['timestamp'] = timestamp



while True:
    # Đọc frame từ camera
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask) # assuming you have 'mask' defined somewhere

    # Gọi mô hình để nhận diện đối tượng trong vùng quan tâm
    results = model(imgRegion, stream=True)

    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Lấy thông tin về hộp giới hạn (bounding box)
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            # Lọc ra các đối tượng là xe và có độ tin cậy lớn hơn 0.3
            if currentClass == 'car' and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Cập nhật thông tin tracking của đối tượng
    resultsTracker = tracker.update(detections)

    # Vẽ đường giới hạn trên ảnh
    cv2.line(img, (first_limit[0], first_limit[1]), (first_limit[2], first_limit[3]), (0, 0, 255), 5)
    cv2.line(img, (last_limit[0], last_limit[1]), (last_limit[2], last_limit[3]), (0, 0, 255), 5)

    # Hiển thị thông tin vận tốc và tính toán vận tốc trung bình
    for result in resultsTracker:
        x1, y1, x2, y2, object_id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1

        # Vẽ khung của đối tượng
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        # Lấy thông tin vận tốc
        speed = calculate_speed((first_limit[0] + first_limit[2])/2, first_limit[1],(last_limit[0] + last_limit[2])/2, last_limit[1])

        # Hiển thị thông tin vận tốc
        speed_text = f'{speed:.2f} km/h'
        cvzone.putTextRect(img, speed_text, (max(0, x1), max(35, y1)),
                        scale=1, thickness=2, offset=10, colorR=(255, 0, 0))

        # Cập nhật và tính toán vận tốc trung bình
        update_object_info(object_id, time.time(), speed)

    # Hiển thị ảnh và chờ nhấn phím
    cv2.imshow("Image", img)
    cv2.waitKey(1)



0: 384x640 2 cars, 1 truck, 529.4ms
Speed: 3.0ms preprocess, 529.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 574.0ms
Speed: 3.4ms preprocess, 574.0ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 trucks, 522.5ms
Speed: 1.4ms preprocess, 522.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 trucks, 501.0ms
Speed: 6.0ms preprocess, 501.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 trucks, 500.3ms
Speed: 5.2ms preprocess, 500.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 trucks, 507.1ms
Speed: 0.0ms preprocess, 507.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 2 trucks, 522.0ms
Speed: 4.2ms preprocess, 522.0ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 500.1ms
Speed: 0.6ms preprocess,

KeyboardInterrupt: 

In [15]:
# Khởi tạo đối tượng tracker với các tham số
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# Định nghĩa giới hạn theo tọa độ [x1, y1, x2, y2]
first_limit = [300, 340, 700, 340]
last_limit = [0, 597, 673, 597]

# Danh sách lưu trữ các đối tượng đã đếm
total_objects = {}
start = {}

def calculate_speed(x1, y1, x2, y2, time_interval):
    # Thực hiện tính toán vận tốc dựa trên tọa độ (x1, y1) và (x2, y2)
    # Bạn có thể sử dụng các phương pháp tính toán vận tốc, ví dụ: khoảng cách / thời gian
    # Đây chỉ là một hàm giả định, bạn cần thay thế bằng cách tính toán thực tế của bạn
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    # Giả sử thời gian giữa hai frame là 1 giây
    speed = distance / time_interval
    return speed

def update_object_info(object_id, timestamp, speed):
    if object_id not in total_objects:
        total_objects[object_id] = {'speed': None, 'timestamp': None}
    total_objects[object_id]['speed'] = speed
    total_objects[object_id]['timestamp'] = timestamp

while True:
    # Đọc frame từ camera
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask) # assuming you have 'mask' defined somewhere

    # Gọi mô hình để nhận diện đối tượng trong vùng quan tâm
    results = model(imgRegion, stream=True)

    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Lấy thông tin về hộp giới hạn (bounding box)
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            # Lọc ra các đối tượng là xe và có độ tin cậy lớn hơn 0.3
            if currentClass == 'car' and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Cập nhật thông tin tracking của đối tượng
    resultsTracker = tracker.update(detections)

    # Vẽ đường giới hạn trên ảnh
    cv2.line(img, (first_limit[0], first_limit[1]), (first_limit[2], first_limit[3]), (0, 0, 255), 5)
    cv2.line(img, (last_limit[0], last_limit[1]), (last_limit[2], last_limit[3]), (0, 0, 255), 5)

    # Hiển thị thông tin vận tốc và tính toán vận tốc trung bình
    for result in resultsTracker:
        x1, y1, x2, y2, object_id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1
        cx, cy = x1 + w // 2, y1 + h // 2
        if first_limit[0] < cx < first_limit[2] and first_limit[1] - 15 < cy < first_limit[1] + 15:
            start[object_id] = time.time()

        if last_limit[0] < cx < last_limit[2] and last_limit[1] - 15 < cy < last_limit[1] + 15:
            if object_id in start:
                elapsed_time = time.time() - start[object_id]
                if elapsed_time > 0:
                    # Vẽ khung của đối tượng
                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 2)
                    # Lấy thông tin vận tốc
                    speed = calculate_speed(x1, y1, x2, y2, elapsed_time)
                    # Hiển thị thông tin vận tốc
                    speed_text = f'{speed:.2f} km/h'
                    cv2.putText(img, speed_text, (max(0, x1), max(35, y1)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
                    # Cập nhật và tính toán vận tốc trung bình
                    update_object_info(object_id, time.time(), speed)

    # Hiển thị ảnh và chờ nhấn phím
    cv2.imshow("Image", img)
    cv2.waitKey(1)


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'
